# What's this??
Seleniumを使って三井の賃貸のサイトからデータを引っ張ってくる

https://mfhl.mitsui-chintai.co.jp/chintai/index.html

In [1]:
# import libraries
import time
import pandas as pd
import numpy as np
import os
import urllib.request

In [2]:
# import selenium libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# import original libraries
from utils.python import mitsui

In [4]:
# variables
path = '/Users/ryusukekimura/Google ドライブ/github/demonstration/rpa/chrome-driver-79.0.3945.36/chromedriver'
url = 'https://mfhl.mitsui-chintai.co.jp/chintai/index.html'
timeout = 10

In [5]:
# create output directory
dir_data = './data/mitsui/'
if not os.path.isdir(dir_data):
    os.makedirs(dir_data)
dir_img = './img/mitsui/'
if not os.path.isdir(dir_img):
    os.makedirs(dir_img)

In [6]:
# define output
cols =  ['bld_name', 'room_name', 'room_madori', 'bld_address', 'access', 'num_rooms',
              'chinryo', 'kanrihi', 'sikikin', 'reikin', 'kibo_kozo', 'menseki_tsubo', 'syunko_nengappi', 'keiyaku_kikan', 'kousinryo',
               'torihiki_taiyo', 'parking', 'hoken', 'kanri_gaisya', 'kagi_koukan', 'kouza_furikae', 'hosyo_gaisya', 'setsubi', 'bikou']
df = pd.DataFrame(index=[], columns=[cols])

In [7]:
# driver settings
driver = webdriver.Chrome(executable_path=path)

In [8]:
# open top page
driver.get(url)

In [9]:
# move to search page
xpath = '//*[@id="main"]/div[1]/div[1]/div[2]/div[1]/a'
driver.find_element_by_xpath(xpath).click()
driver.implicitly_wait(timeout)

In [11]:
# move to search result page
xpath = '//*[@id="active"]'
driver.find_element_by_xpath(xpath).click()
time.sleep(timeout)

In [12]:
# get number of search result pages
xpath = '//*[@id="container"]/section/div[2]/div[2]/div[1]/span[1]'
num_pages = int(driver.find_element_by_xpath(xpath).text)

In [13]:
# for each page
for page in range(1, num_pages + 1):
    print('|-- processing page ' + str(page) + ' ...')
    
    for bukken in range(1, 11):
        print('   |-- bukken number ' + str(bukken) + ' ...')
        
        # extract room information
        xpath = '//*[@id="container"]/section/li[' + str(bukken) + ']/div[2]/div[1]/div/h3'
        driver.find_element_by_xpath(xpath).click()
        time.sleep(timeout)
        array = mitsui.extract_information(driver)
        
        # add array to dataframe
        record = pd.Series(array, index=df.columns)
        df = df.append(record, ignore_index=True)

        # go back to search result pages
        driver.back()
        time.sleep(timeout)
        
    # move to next page
    if page != num_pages:
        xpath = '//*[@id="container"]/section/div[1]/div/div/div[2]/div[2]/div[4]/ul/li[2]/a'
        driver.find_element_by_xpath(xpath).click()
        driver.implicitly_wait(timeout)
        
driver.back()
time.sleep(timeout)

|-- processing page 1 ...
   |-- bukken number 1 ...
   |-- bukken number 2 ...
   |-- bukken number 3 ...
   |-- bukken number 4 ...
   |-- bukken number 5 ...
   |-- bukken number 6 ...
   |-- bukken number 7 ...
   |-- bukken number 8 ...
   |-- bukken number 9 ...
   |-- bukken number 10 ...
|-- processing page 2 ...
   |-- bukken number 1 ...
   |-- bukken number 2 ...
   |-- bukken number 3 ...
   |-- bukken number 4 ...
   |-- bukken number 5 ...
   |-- bukken number 6 ...
   |-- bukken number 7 ...
   |-- bukken number 8 ...
   |-- bukken number 9 ...
   |-- bukken number 10 ...
|-- processing page 3 ...
   |-- bukken number 1 ...
   |-- bukken number 2 ...
   |-- bukken number 3 ...
   |-- bukken number 4 ...
   |-- bukken number 5 ...
   |-- bukken number 6 ...
   |-- bukken number 7 ...


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container"]/section/div[1]/div/div/div[1]/table[1]/tbody/tr[1]/td"}
  (Session info: chrome=80.0.3987.87)


In [ ]:
driver.back()

In [ ]:
# print output
df

In [ ]:
# close driver
#driver.close()

In [ ]:
# output
file = './data/mitsui/output.csv'
df.to_csv(file, index=False)